In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import  mysql.connector as conn
from flask import Flask , jsonify



In [2]:
headers = {'User Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

In [3]:
url = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'

In [4]:
url

'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'

In [5]:
res = requests.get(url,headers=headers)

In [6]:
print(res.status_code)

200


In [7]:
soup = bs(res.content,'html.parser')



In [8]:
#creating list for data to be fetched 
Movie_name =[]
Year_of_release = []
Duration =[]
Imdb_rating = []
Genre = []
Director= []
Starcast = []
Synopsis = []

In [9]:
movie_data = soup.find_all('div',{'class':'lister-item mode-advanced'})

In [10]:
for data in movie_data:
    name = data.h3.a.text
    Movie_name.append(name)
    
    yr = data.h3.find('span',{'class':'lister-item-year text-muted unbold'}).text.replace('(','').replace(')','')
    Year_of_release.append(yr)
    
    runtime = data.p.find('span',{'class':'runtime'}).text.replace(' min','')
    Duration.append(runtime)
    
    rating = data.find('div',{'class':'inline-block ratings-imdb-rating'}).text.strip()
    Imdb_rating.append(rating)
    
    gen = data.p.find('span',{'class':'genre'}).text.strip()
    Genre.append(gen)
    
    direct = data.find('p',{'class':''}).text.strip().replace('\n','').split('|')
    direct = [d.strip() for d in direct]
    direct = [direct[i].replace(j,'')for i,j in enumerate(['Director:','Stars:'])]
    Director.append(direct[0])
    Starcast.append([s.strip() for s in direct[1].split(',')])
    
    des = data.find_all('p', class_ = 'text-muted')[1].text.replace('\n','')
    Synopsis.append(des)
    
    
    
    
    
    

In [11]:
# using pandas to convert into data frame

dict_ ={'Name':Movie_name,'Release_Year':Year_of_release,'Descrition':Synopsis,'Runtime':Duration,'IMDB_Rating':Imdb_rating,'Genre_':Genre,'Directors':Director,'Cast':Starcast}

df = pd.DataFrame.from_dict(dict_, orient='index')
df = df.transpose()

In [12]:
df

,Name,Release_Year,Descrition,Runtime,IMDB_Rating,Genre_,Directors,Cast
0,The Shawshank Redemption,1994,Two imprisoned men bond over a number of years...,142,9.3,Drama,Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
1,The Godfather,1972,The aging patriarch of an organized crime dyna...,175,9.2,"Crime, Drama",Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K..."
2,Rocketry: The Nambi Effect,2022,Based on the life of Indian Space Research Org...,157,9.0,"Biography, Drama",Madhavan,"[Madhavan, Simran, Rajit Kapoor, Ravi Raghaven..."
3,The Dark Knight,2008,When the menace known as the Joker wreaks havo...,152,9.0,"Action, Crime, Drama",Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
4,The Lord of the Rings: The Return of the King,2003,Gandalf and Aragorn lead the World of Men agai...,201,9.0,"Action, Adventure, Drama",Peter Jackson,"[Elijah Wood, Viggo Mortensen, Ian McKellen, O..."
...,...,...,...,...,...,...,...,...
95,Hamilton,2020,The real life of one of America's foremost fou...,160,8.3,"Biography, Drama, History",Thomas Kail,"[Lin-Manuel Miranda, Phillipa Soo, Leslie Odom..."
96,Ratsasan,2018,A sub-inspector sets out in pursuit of a myste...,170,8.3,"Crime, Drama, Mystery",Ram Kumar,"[Vishnu Vishal, Amala Paul, Radha Ravi, Sangil..."
97,Ayla: The Daughter of War,2017,"In 1950, amid-st the ravages of the Korean War...",125,8.3,"Biography, Drama, History",Can Ulkay,"[Çetin Tekindor, Ismail Hacioglu, Kyung-jin Le..."
98,Vikram Vedha,2017,"Vikram, a no-nonsense police officer, accompan...",147,8.3,"Action, Crime, Drama","Directors:Gayatri, Pushkar, Gowtham Selvaraj","[Madhavan, Vijay Sethupathi, Shraddha Srinath,..."


In [13]:
df.to_csv('IMDB_Data.csv')

In [14]:
app = Flask(__name__)

mydb = conn.connect(host = "localhost" , user = 'root' , passwd = "password")
cursor = mydb.cursor()
cursor.execute("create database if not exists imdb")


In [15]:
!csvsql --db mysql://root:<password>@localhost:3306/imdb --insert IMDB_Data.csv #ignore the warning as table was created and i re-run the command.

c:\users\anile\anaconda3\lib\site-packages\win32\lib\pywintypes.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
c:\users\anile\anaconda3\lib\site-packages\agate\utils.py:285: UnnamedColumnWarning: Column 0 has no name. Using "a".
OperationalError: (MySQLdb.OperationalError) (1050, "Table 'imdb_data' already exists")
[SQL: 
CREATE TABLE `IMDB_Data` (
	a DECIMAL(38, 0) NOT NULL, 
	`Name` VARCHAR(68) NOT NULL, 
	`Release_Year` VARCHAR(7) NOT NULL, 
	`Descrition` VARCHAR(239) NOT NULL, 
	`Runtime` DECIMAL(38, 0) NOT NULL, 
	`IMDB_Rating` DECIMAL(38, 1) NOT NULL, 
	`Genre_` VARCHAR(28) NOT NULL, 
	`Directors` VARCHAR(55) NOT NULL, 
	`Cast` VARCHAR(90) NOT NULL
)

]
(Background on this error at: http://sqlalche.me/e/14/e3q8)


In [ ]:
@app.route("/fetchdata",methods = ['POST','GET'])
def fetch_data():
    cursor.execute("select * from imdb.imdb_data")
    for i in cursor.fetchall():
        return jsonify(str(i))
    
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Sep/2022 13:41:09] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [16/Sep/2022 13:41:17] "GET /fetchdata HTTP/1.1" 200 -


In [ ]:
#data shown on browser